In [1]:
# Import der libraries
import sqlite3 as sq
from sqlite3 import Error
import pandas as pd
import requests

#Verbindung mit Google Drive
from google.colab import drive 
drive.mount("/content/gdrive", force_remount=True)

#Initialisierung von Sqlite
conn = sq.connect('quelldatenbank.db')
cur = conn.cursor()

# Erstellen der Tabellen
sql_create_source_data = requests.get('https://raw.githubusercontent.com/Fuenfgeld/DMA2022TeamA/main/Create_statements.sql').text
cur.executescript(sql_create_source_data)
conn.commit()
print("Tabellen erfolgreich erstellt")

# Pfade definieren
file_path = '/content/gdrive/Shareddrives/Gruppe A/csv_data/'
disease = ['allergy', 'asthma', 'breast_cancer', 'colorectal_cancer', 'covid19', 'dermatitis', 'lung_cancer', 'metabolic_syndrome_disease']
dataset_file = ['patients', 'organizations', 'providers', 'payers', 'payer_transitions', 'encounters', 'careplans', 'conditions', 'devices', 'disease', 'imaging_studies', 'immunizations', 'observations', 'procedures', 'supplies', 'medications']

#  .csv Dateien auslesen und in Datenbank schreiben + Hinzufügen Spalte "dataset_origin"
for actual_disease in disease:
  actual_file_path = file_path+actual_disease+'/'
  for actual_dataset_file in dataset_file:
    actual_insert_df = pd.read_csv(actual_file_path+actual_dataset_file+'.csv')
    actual_insert_df = actual_insert_df.rename(columns={'PATIENT': 'patient_id', 'PAYER': 'payer_id', 'ENCOUNTER': 'encounter_id', 'ORGANIZATION': 'organization_id', 'PROVIDER': 'provider_id'})
    actual_insert_df.insert(loc=len(actual_insert_df.columns), column='dataset_origin', value=actual_disease)
    actual_insert_df.to_sql(actual_dataset_file, conn, if_exists='append', index=False)
conn.commit()
print("Daten erfolgreich eingefügt")







Mounted at /content/gdrive
